In [1]:
import datetime
import pandas as pd
import warnings
import sklearn.externals as extjoblib
import joblib
warnings.filterwarnings("ignore")

#from google.cloud import storage

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score

from catboost import CatBoostClassifier, Pool
#from sklearn.externals import joblib
from sklearn.pipeline import Pipeline

In [2]:
RANDOM_SEED = 42

In [3]:
df_proc = pd.read_csv('cleaned_non-vectorized_data.csv')
df_proc = df_proc[~df_proc['Text'].isna()]
X = df_proc['Text']

In [4]:
df_proc

,Text,tokens,disgust,joy,anger,surprised,sad,fear,neutral
0,come mert ’ today let u take care lunch enjoy ...,"['come', 'mert', '’', 'today', 'let', 'u', 'ta...",0,0,0,0,0,0,1
1,nxt gt lay 20 staff tech 's latest cutback rb_...,"['nxt', 'gt', 'lay', '20', 'staff', 'tech', ""'...",0,0,0,0,0,0,1
2,layoff 20 workforce 100 employee sf bay area,"['layoff', '20', 'workforce', '100', 'employee...",0,0,0,0,0,0,1
3,today ’ lunch special smoked pork sausage onio...,"['today', '’', 'lunch', 'special', 'smoked', '...",0,0,0,0,0,0,1
4,come mert ’ today grab salmon cake two home co...,"['come', 'mert', '’', 'today', 'grab', 'salmon...",0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
9277,traik01 cdc people warmed u 2 year ago .... sa...,"['traik01', 'cdc', 'people', 'warmed', 'u', '2...",0,0,0,0,1,0,0
9278,sorry ’ promo code share lately 😭 promos autom...,"['sorry', '’', 'promo', 'code', 'share', 'late...",0,0,0,0,1,0,0
9279,poor lad,"['poor', 'lad']",0,0,0,0,1,0,0
9280,one day able bill order tmobile bill sadly tod...,"['one', 'day', 'able', 'bill', 'order', 'tmobi...",0,0,0,0,1,0,0


In [5]:
y = df_proc.loc[:,['disgust', 'joy', 'anger', 'surprised', 'sad', 'fear', 'neutral']]

In [6]:
y

,disgust,joy,anger,surprised,sad,fear,neutral
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...
9277,0,0,0,0,1,0,0
9278,0,0,0,0,1,0,0
9279,0,0,0,0,1,0,0
9280,0,0,0,0,1,0,0


In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

### Preprocessing - BOW Approach

In [8]:
vectorizer = TfidfVectorizer(lowercase=True, 
                             stop_words='english', 
                             ngram_range=(1,2),
                             max_df=0.75,
                             min_df=50,
                             norm='l1')

In [10]:
clf = CatBoostClassifier(random_state=RANDOM_SEED)
multi_out_clf = MultiOutputClassifier(clf)

In [11]:
# Transform the features and fit them to the classifier
# multi_out_clf.fit(transformer.fit_transform(vectorizer.fit_transform(X_train)), y_train)

In [12]:
pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('multi-classifier', multi_out_clf)
])

In [13]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vectorizer', 'multi-classifier', 'vectorizer__analyzer', 'vectorizer__binary', 'vectorizer__decode_error', 'vectorizer__dtype', 'vectorizer__encoding', 'vectorizer__input', 'vectorizer__lowercase', 'vectorizer__max_df', 'vectorizer__max_features', 'vectorizer__min_df', 'vectorizer__ngram_range', 'vectorizer__norm', 'vectorizer__preprocessor', 'vectorizer__smooth_idf', 'vectorizer__stop_words', 'vectorizer__strip_accents', 'vectorizer__sublinear_tf', 'vectorizer__token_pattern', 'vectorizer__tokenizer', 'vectorizer__use_idf', 'vectorizer__vocabulary', 'multi-classifier__estimator__random_state', 'multi-classifier__estimator', 'multi-classifier__n_jobs'])

In [16]:
parameters = {
    "vectorizer__max_df": (0.5, 1.0),
    "vectorizer__min_df": (50, 100),
    "vectorizer__ngram_range": ((1, 1), (1, 2)),
}


In [17]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Learning rate set to 0.024188
0:	learn: 0.6821824	total: 72.6ms	remaining: 1m 12s
1:	learn: 0.6720551	total: 86.3ms	remaining: 43.1s
2:	learn: 0.6618781	total: 97.8ms	remaining: 32.5s
3:	learn: 0.6522894	total: 109ms	remaining: 27.2s
4:	learn: 0.6425813	total: 123ms	remaining: 24.4s
5:	learn: 0.6335541	total: 134ms	remaining: 22.2s
6:	learn: 0.6255904	total: 146ms	remaining: 20.8s
7:	learn: 0.6181819	total: 158ms	remaining: 19.6s
8:	learn: 0.6107568	total: 170ms	remaining: 18.7s
9:	learn: 0.6041437	total: 183ms	remaining: 18.1s
10:	learn: 0.5976314	total: 195ms	remaining: 17.5s
11:	learn: 0.5912661	total: 208ms	remaining: 17.1s
12:	learn: 0.5853599	total: 224ms	remaining: 17s
13:	learn: 0.5794305	total: 239ms	remaining: 16.8s
14:	learn: 0.5738456	total: 253ms	remaining: 16.6s
15:	learn: 0.5687992	total: 267ms	remaining: 16.4s
16:	learn: 0.5642245	total: 281ms	remaining: 16.3s
17:	learn: 0.5575961	total: 294ms	remaining: 16s
18

171:	learn: 0.4182438	total: 2.45s	remaining: 11.8s
172:	learn: 0.4179772	total: 2.47s	remaining: 11.8s
173:	learn: 0.4177916	total: 2.48s	remaining: 11.8s
174:	learn: 0.4175674	total: 2.5s	remaining: 11.8s
175:	learn: 0.4173577	total: 2.51s	remaining: 11.8s
176:	learn: 0.4171548	total: 2.52s	remaining: 11.7s
177:	learn: 0.4169821	total: 2.54s	remaining: 11.7s
178:	learn: 0.4167820	total: 2.55s	remaining: 11.7s
179:	learn: 0.4165388	total: 2.57s	remaining: 11.7s
180:	learn: 0.4163590	total: 2.58s	remaining: 11.7s
181:	learn: 0.4161042	total: 2.59s	remaining: 11.6s
182:	learn: 0.4159312	total: 2.6s	remaining: 11.6s
183:	learn: 0.4157111	total: 2.62s	remaining: 11.6s
184:	learn: 0.4154500	total: 2.63s	remaining: 11.6s
185:	learn: 0.4152321	total: 2.64s	remaining: 11.6s
186:	learn: 0.4150411	total: 2.66s	remaining: 11.6s
187:	learn: 0.4147682	total: 2.67s	remaining: 11.5s
188:	learn: 0.4145820	total: 2.68s	remaining: 11.5s
189:	learn: 0.4143636	total: 2.7s	remaining: 11.5s
190:	learn: 0.4

336:	learn: 0.3913656	total: 4.71s	remaining: 9.27s
337:	learn: 0.3912602	total: 4.72s	remaining: 9.26s
338:	learn: 0.3911134	total: 4.74s	remaining: 9.24s
339:	learn: 0.3909887	total: 4.75s	remaining: 9.22s
340:	learn: 0.3909081	total: 4.76s	remaining: 9.21s
341:	learn: 0.3907748	total: 4.78s	remaining: 9.2s
342:	learn: 0.3906228	total: 4.79s	remaining: 9.18s
343:	learn: 0.3904175	total: 4.81s	remaining: 9.17s
344:	learn: 0.3903285	total: 4.82s	remaining: 9.16s
345:	learn: 0.3902194	total: 4.84s	remaining: 9.15s
346:	learn: 0.3901068	total: 4.86s	remaining: 9.14s
347:	learn: 0.3900023	total: 4.87s	remaining: 9.12s
348:	learn: 0.3898602	total: 4.88s	remaining: 9.11s
349:	learn: 0.3897632	total: 4.89s	remaining: 9.09s
350:	learn: 0.3896873	total: 4.91s	remaining: 9.08s
351:	learn: 0.3895361	total: 4.92s	remaining: 9.06s
352:	learn: 0.3893711	total: 4.94s	remaining: 9.06s
353:	learn: 0.3892193	total: 4.96s	remaining: 9.04s
354:	learn: 0.3891034	total: 4.97s	remaining: 9.03s
355:	learn: 0

507:	learn: 0.3679306	total: 7.14s	remaining: 6.91s
508:	learn: 0.3678076	total: 7.16s	remaining: 6.91s
509:	learn: 0.3677163	total: 7.17s	remaining: 6.89s
510:	learn: 0.3675613	total: 7.19s	remaining: 6.88s
511:	learn: 0.3674065	total: 7.2s	remaining: 6.86s
512:	learn: 0.3672851	total: 7.21s	remaining: 6.85s
513:	learn: 0.3671896	total: 7.23s	remaining: 6.83s
514:	learn: 0.3670119	total: 7.24s	remaining: 6.82s
515:	learn: 0.3669336	total: 7.26s	remaining: 6.81s
516:	learn: 0.3667192	total: 7.27s	remaining: 6.79s
517:	learn: 0.3666005	total: 7.28s	remaining: 6.78s
518:	learn: 0.3664292	total: 7.3s	remaining: 6.76s
519:	learn: 0.3663120	total: 7.31s	remaining: 6.75s
520:	learn: 0.3661592	total: 7.32s	remaining: 6.73s
521:	learn: 0.3660401	total: 7.33s	remaining: 6.72s
522:	learn: 0.3659152	total: 7.35s	remaining: 6.71s
523:	learn: 0.3657453	total: 7.37s	remaining: 6.69s
524:	learn: 0.3656478	total: 7.38s	remaining: 6.68s
525:	learn: 0.3654785	total: 7.39s	remaining: 6.66s
526:	learn: 0.

668:	learn: 0.3488112	total: 9.38s	remaining: 4.64s
669:	learn: 0.3487923	total: 9.4s	remaining: 4.63s
670:	learn: 0.3487024	total: 9.41s	remaining: 4.61s
671:	learn: 0.3485306	total: 9.42s	remaining: 4.6s
672:	learn: 0.3483959	total: 9.44s	remaining: 4.59s
673:	learn: 0.3483089	total: 9.45s	remaining: 4.57s
674:	learn: 0.3481971	total: 9.46s	remaining: 4.56s
675:	learn: 0.3481761	total: 9.48s	remaining: 4.54s
676:	learn: 0.3481569	total: 9.49s	remaining: 4.53s
677:	learn: 0.3480760	total: 9.51s	remaining: 4.52s
678:	learn: 0.3479523	total: 9.52s	remaining: 4.5s
679:	learn: 0.3477874	total: 9.54s	remaining: 4.49s
680:	learn: 0.3477461	total: 9.55s	remaining: 4.47s
681:	learn: 0.3476583	total: 9.56s	remaining: 4.46s
682:	learn: 0.3475011	total: 9.58s	remaining: 4.45s
683:	learn: 0.3474079	total: 9.59s	remaining: 4.43s
684:	learn: 0.3472861	total: 9.61s	remaining: 4.42s
685:	learn: 0.3472676	total: 9.62s	remaining: 4.4s
686:	learn: 0.3472509	total: 9.64s	remaining: 4.39s
687:	learn: 0.34

...

435:	learn: 0.5383157	total: 5.28s	remaining: 6.83s
436:	learn: 0.5380708	total: 5.29s	remaining: 6.82s
437:	learn: 0.5378319	total: 5.3s	remaining: 6.81s
438:	learn: 0.5376369	total: 5.32s	remaining: 6.79s
439:	learn: 0.5374288	total: 5.33s	remaining: 6.78s
440:	learn: 0.5372098	total: 5.34s	remaining: 6.77s
441:	learn: 0.5370945	total: 5.35s	remaining: 6.75s
442:	learn: 0.5368901	total: 5.36s	remaining: 6.74s
443:	learn: 0.5365270	total: 5.37s	remaining: 6.73s
444:	learn: 0.5363769	total: 5.38s	remaining: 6.71s
445:	learn: 0.5361328	total: 5.39s	remaining: 6.7s
446:	learn: 0.5358937	total: 5.41s	remaining: 6.69s
447:	learn: 0.5357140	total: 5.42s	remaining: 6.67s
448:	learn: 0.5355481	total: 5.43s	remaining: 6.66s
449:	learn: 0.5353656	total: 5.44s	remaining: 6.65s
450:	learn: 0.5351997	total: 5.45s	remaining: 6.63s
451:	learn: 0.5350700	total: 5.46s	remaining: 6.62s
452:	learn: 0.5348900	total: 5.47s	remaining: 6.61s
453:	learn: 0.5347046	total: 5.48s	remaining: 6.6s
454:	learn: 0.5

609:	learn: 0.5062031	total: 7.54s	remaining: 4.82s
610:	learn: 0.5060708	total: 7.55s	remaining: 4.81s
611:	learn: 0.5058840	total: 7.56s	remaining: 4.79s
612:	learn: 0.5056824	total: 7.58s	remaining: 4.78s
613:	learn: 0.5055028	total: 7.59s	remaining: 4.77s
614:	learn: 0.5053586	total: 7.6s	remaining: 4.76s
615:	learn: 0.5052245	total: 7.61s	remaining: 4.74s
616:	learn: 0.5050384	total: 7.62s	remaining: 4.73s
617:	learn: 0.5049054	total: 7.63s	remaining: 4.72s
618:	learn: 0.5047595	total: 7.65s	remaining: 4.71s
619:	learn: 0.5046240	total: 7.66s	remaining: 4.69s
620:	learn: 0.5044478	total: 7.67s	remaining: 4.68s
621:	learn: 0.5043365	total: 7.68s	remaining: 4.67s
622:	learn: 0.5041722	total: 7.69s	remaining: 4.66s
623:	learn: 0.5039929	total: 7.71s	remaining: 4.64s
624:	learn: 0.5038410	total: 7.72s	remaining: 4.63s
625:	learn: 0.5036774	total: 7.73s	remaining: 4.62s
626:	learn: 0.5035115	total: 7.74s	remaining: 4.61s
627:	learn: 0.5033757	total: 7.76s	remaining: 4.59s
628:	learn: 0

779:	learn: 0.4820764	total: 9.59s	remaining: 2.7s
780:	learn: 0.4819558	total: 9.6s	remaining: 2.69s
781:	learn: 0.4818378	total: 9.62s	remaining: 2.68s
782:	learn: 0.4816752	total: 9.63s	remaining: 2.67s
783:	learn: 0.4815626	total: 9.64s	remaining: 2.65s
784:	learn: 0.4814209	total: 9.65s	remaining: 2.64s
785:	learn: 0.4812418	total: 9.66s	remaining: 2.63s
786:	learn: 0.4811432	total: 9.68s	remaining: 2.62s
787:	learn: 0.4810356	total: 9.69s	remaining: 2.61s
788:	learn: 0.4809555	total: 9.7s	remaining: 2.59s
789:	learn: 0.4808930	total: 9.71s	remaining: 2.58s
790:	learn: 0.4807955	total: 9.72s	remaining: 2.57s
791:	learn: 0.4806793	total: 9.74s	remaining: 2.56s
792:	learn: 0.4805497	total: 9.75s	remaining: 2.54s
793:	learn: 0.4804419	total: 9.76s	remaining: 2.53s
794:	learn: 0.4803249	total: 9.77s	remaining: 2.52s
795:	learn: 0.4801969	total: 9.79s	remaining: 2.51s
796:	learn: 0.4800638	total: 9.8s	remaining: 2.5s
797:	learn: 0.4799206	total: 9.81s	remaining: 2.48s
798:	learn: 0.479

950:	learn: 0.4621253	total: 11.6s	remaining: 600ms
951:	learn: 0.4620223	total: 11.7s	remaining: 588ms
952:	learn: 0.4619365	total: 11.7s	remaining: 576ms
953:	learn: 0.4617820	total: 11.7s	remaining: 563ms
954:	learn: 0.4616629	total: 11.7s	remaining: 551ms
955:	learn: 0.4615696	total: 11.7s	remaining: 539ms
956:	learn: 0.4614526	total: 11.7s	remaining: 527ms
957:	learn: 0.4613903	total: 11.7s	remaining: 514ms
958:	learn: 0.4612728	total: 11.7s	remaining: 502ms
959:	learn: 0.4611220	total: 11.8s	remaining: 490ms
960:	learn: 0.4610546	total: 11.8s	remaining: 478ms
961:	learn: 0.4609340	total: 11.8s	remaining: 465ms
962:	learn: 0.4608373	total: 11.8s	remaining: 453ms
963:	learn: 0.4607259	total: 11.8s	remaining: 441ms
964:	learn: 0.4605957	total: 11.8s	remaining: 429ms
965:	learn: 0.4604802	total: 11.8s	remaining: 416ms
966:	learn: 0.4603479	total: 11.8s	remaining: 404ms
967:	learn: 0.4602460	total: 11.9s	remaining: 392ms
968:	learn: 0.4601241	total: 11.9s	remaining: 380ms
969:	learn: 

GridSearchCV(estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(max_df=0.75, min_df=50,
                                                        ngram_range=(1, 2),
                                                        norm='l1',
                                                        stop_words='english')),
                                       ('multi-classifier',
                                        MultiOutputClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x7ff492e926a0>))]),
             n_jobs=-1,
             param_grid={'vectorizer__max_df': (0.5, 1.0),
                         'vectorizer__min_df': (50, 100),
                         'vectorizer__ngram_range': ((1, 1), (1, 2))},
             verbose=1)

In [18]:
best_parameters = grid_search.best_estimator_.get_params()

In [19]:
best_parameters

{'memory': None,
 'steps': [('vectorizer',
   TfidfVectorizer(max_df=0.5, min_df=50, ngram_range=(1, 2), norm='l1',
                   stop_words='english')),
  ('multi-classifier',
   MultiOutputClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x7ff492292700>))],
 'verbose': False,
 'vectorizer': TfidfVectorizer(max_df=0.5, min_df=50, ngram_range=(1, 2), norm='l1',
                 stop_words='english'),
 'multi-classifier': MultiOutputClassifier(estimator=<catboost.core.CatBoostClassifier object at 0x7ff492292700>),
 'vectorizer__analyzer': 'word',
 'vectorizer__binary': False,
 'vectorizer__decode_error': 'strict',
 'vectorizer__dtype': numpy.float64,
 'vectorizer__encoding': 'utf-8',
 'vectorizer__input': 'content',
 'vectorizer__lowercase': True,
 'vectorizer__max_df': 0.5,
 'vectorizer__max_features': None,
 'vectorizer__min_df': 50,
 'vectorizer__ngram_range': (1, 2),
 'vectorizer__norm': 'l1',
 'vectorizer__preprocessor': None,
 'vectorizer__smooth_idf': True,
 

In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score

def print_classification_scores(y_test, pred):
    print('Accuracy Score:',accuracy_score(y_test, pred))
    print('Precision Score:',precision_score(y_test, pred, average='micro'))
    print('Recall Score:',recall_score(y_test, pred, average='micro'))
    print('F1 Score:',f1_score(y_test, pred, average='micro'))
    print('AUC Score:',roc_auc_score(y_test, pred, average='micro'))

In [21]:
pred = grid_search.predict(X_test)
print_classification_scores(y_test, pred)

Accuracy Score: 0.4750812567713976
Precision Score: 0.7008491182233834
Recall Score: 0.5016362786348761
F1 Score: 0.584741144414169
AUC Score: 0.7295810067940215


### Preprocess - Word Embeddings

In [ ]:
%pip install gensim

In [ ]:
import gzip
import json
import matplotlib.pyplot as plt
import numpy as np
import re
import random
import pandas as pd
import seaborn as sns
import gensim
import nltk
import gensim.downloader
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.word2vec import Word2Vec
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from tqdm import tqdm

In [ ]:
nltk.download('stopwords')

In [ ]:
#https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
stop_words = set(stopwords.words('english'))

In [ ]:
all_tokenized_reviews = []

for review in tqdm(X):
    tokens = [token for token in re.findall(r'\w+', review) if token not in stop_words]
    all_tokenized_reviews.append(tokens)

In [ ]:
full_model = Word2Vec(sentences=all_tokenized_reviews, vector_size=100, 
                      window=2, min_count=100, workers=4, seed=RANDOM_SEED)

In [ ]:
full_model.save("word2vec_tweets.model")

In [ ]:
full_model_kv = full_model.wv

In [ ]:
gnews = gensim.downloader.load('word2vec-google-news-300')
glove_small = gensim.downloader.load('glove-wiki-gigaword-100')
glove_big = gensim.downloader.load('glove-wiki-gigaword-300')

In [ ]:
def generate_dense_features(tokenized_texts, word_vectors): 
    #HINT: Create an empty list to hold your results 
        #HINT:Iterate through each item in tokenized_text
            #HINT:Create a list that contains current item(s) if found in word_vectors
            #HINT:if the length of this list is greater than zero:
                #HINT:We set this as a feature, this is done by using numpy’s mean function and append it to our results list 
            #HINT:Otherwise: create a vector of numpy zeros using word_vectors.vector_size as the parameter and append it to the results list
    #HINT:Return the results list as a numpy array (data type)

    res = []
    for token in tokenized_texts:
        items_in_vocab = [item for item in token if item in word_vectors]
        if len(items_in_vocab) > 0:
            res.append(np.mean(word_vectors[items_in_vocab], axis=0))
        else:
            res.append(np.zeros(word_vectors.vector_size))
    return np.array(res)


In [ ]:
tokenized_train_items = []
for review in tqdm(X_train):
    tokens = [token for token in re.findall(r'\w+', review) if token not in stop_words]
    tokenized_train_items.append(tokens)

In [ ]:
tokenized_test_items = []
for review in tqdm(X_test):
    tokens = [token for token in re.findall(r'\w+', review) if token not in stop_words]
    tokenized_test_items.append(tokens)

In [ ]:
X_train_wp = generate_dense_features(tokenized_train_items, full_model_kv)

In [ ]:
X_test_wp = generate_dense_features(tokenized_test_items, full_model_kv)

In [ ]:
clf = LinearSVC(random_state=RANDOM_SEED)
multi_out_clf = MultiOutputClassifier(clf)


multi_out_clf.fit(X_train_wp, y_train)

In [ ]:
pred = multi_out_clf.predict(X_test_wp)
print_classification_scores(y_test, pred)

Now we can repeat the same process with the pre-trained embeddings

In [ ]:
X_train_wp = generate_dense_features(tokenized_train_items, gnews)

In [ ]:
X_test_wp = generate_dense_features(tokenized_test_items, gnews)

In [ ]:
clf = LinearSVC(random_state=RANDOM_SEED)
multi_out_clf = MultiOutputClassifier(clf)


multi_out_clf.fit(X_train_wp, y_train)

In [ ]:
pred = multi_out_clf.predict(X_test_wp)
print_classification_scores(y_test, pred)

Glove_small

In [ ]:
X_train_wp = generate_dense_features(tokenized_train_items, glove_small)

In [ ]:
X_test_wp = generate_dense_features(tokenized_test_items, glove_small)

In [ ]:
clf = LinearSVC(random_state=RANDOM_SEED)
multi_out_clf = MultiOutputClassifier(clf)


multi_out_clf.fit(X_train_wp, y_train)

In [ ]:
pred = multi_out_clf.predict(X_test_wp)
print_classification_scores(y_test, pred)

Glove_big

In [ ]:
X_train_wp = generate_dense_features(tokenized_train_items, glove_big)

In [ ]:
X_test_wp = generate_dense_features(tokenized_test_items, glove_big)

In [ ]:
clf = LinearSVC(random_state=RANDOM_SEED)
multi_out_clf = MultiOutputClassifier(clf)


multi_out_clf.fit(X_train_wp, y_train)

In [ ]:
pred = multi_out_clf.predict(X_test_wp)
print_classification_scores(y_test, pred)